In [1]:
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from scipy.stats import norm

In [2]:
url = 'https://query1.finance.yahoo.com/v7/finance/download/%5ENSEI?period1=1420070400&period2=1577750400&interval=1d&events=history'
input_file= pd.read_csv(url, header=0, index_col='Date')## header=0 takes the first line as column headers
pd.options.display.max_columns=None
pd.options.display.max_rows=None
input_file.dropna(how='any',inplace=True) #to drop NaN rows and columns
input_file.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,8288.700195,8410.599609,8288.700195,8395.450195,8395.450195,101900.0
2015-01-05,8407.950195,8445.599609,8363.900391,8378.400391,8378.400391,118200.0
2015-01-06,8325.299805,8327.849609,8111.350098,8127.350098,8127.350098,172800.0
2015-01-07,8118.649902,8151.200195,8065.450195,8102.100098,8102.100098,164100.0
2015-01-08,8191.399902,8243.500000,8167.299805,8234.599609,8234.599609,143800.0


### SMA

In [3]:
input_file['SMA50']= input_file['Close'].rolling(50).mean()
input_file['SMA10']=input_file['Close'].rolling(10).mean()

In [4]:
input_file['pos_indicator']=(input_file['SMA10']>input_file['SMA50']) & (input_file['SMA50'].shift(1)>=input_file['SMA10'].shift(1))
input_file['neg_indicator']=(input_file['SMA10']<input_file['SMA50']) & (input_file['SMA50'].shift(1)<=input_file['SMA10'].shift(1))
input_file.tail()

,Open,High,Low,Close,Adj Close,Volume,SMA50,SMA10,pos_indicator,neg_indicator
Date,,,,,,,,,,
2019-12-23,12235.450195,12287.150391,12213.250000,12262.750000,12262.750000,600.0,11883.890996,12106.030078,False,False
2019-12-24,12269.250000,12283.700195,12202.099609,12214.549805,12214.549805,500.0,11903.490996,12141.805078,False,False
2019-12-26,12211.849609,12221.549805,12118.849609,12126.549805,12126.549805,500.0,11919.920996,12163.445020,False,False
2019-12-27,12172.900391,12258.450195,12157.900391,12245.799805,12245.799805,400.0,11938.013984,12190.845020,False,False
2019-12-30,12274.900391,12286.450195,12213.799805,12255.849609,12255.849609,400.0,11954.564980,12207.759961,False,False


### Bollinger bands

In [5]:
input_file['SMA20']= input_file['Close'].rolling(20).mean()
input_file['StdDev']= input_file['Close'].rolling(20).std()
input_file['UpperBand']= input_file['SMA20']+2*input_file['StdDev']
input_file['LowerBand']=input_file['SMA20']-2*input_file['StdDev']

In [6]:
input_file['BBpos']=(input_file['Close']>input_file['UpperBand'])
input_file['BBneg']=input_file['Close']<input_file['LowerBand']
input_file.tail()

,Open,High,Low,Close,Adj Close,Volume,SMA50,SMA10,pos_indicator,neg_indicator,SMA20,StdDev,UpperBand,LowerBand,BBpos,BBneg
Date,,,,,,,,,,,,,,,,
2019-12-23,12235.450195,12287.150391,12213.250000,12262.750000,12262.750000,600.0,11883.890996,12106.030078,False,False,12068.445117,122.474107,12313.393332,11823.496903,False,False
2019-12-24,12269.250000,12283.700195,12202.099609,12214.549805,12214.549805,500.0,11903.490996,12141.805078,False,False,12077.287598,126.456935,12330.201468,11824.373728,False,False
2019-12-26,12211.849609,12221.549805,12118.849609,12126.549805,12126.549805,500.0,11919.920996,12163.445020,False,False,12078.580078,126.840343,12332.260763,11824.899393,False,False
2019-12-27,12172.900391,12258.450195,12157.900391,12245.799805,12245.799805,400.0,11938.013984,12190.845020,False,False,12083.312549,131.375124,12346.062796,11820.562302,False,False
2019-12-30,12274.900391,12286.450195,12213.799805,12255.849609,12255.849609,400.0,11954.564980,12207.759961,False,False,12093.302539,136.682271,12366.667081,11819.937997,False,False


### MACD

In [7]:
input_file['EMA12'] = input_file['Close'].ewm(span=12).mean()
input_file['EMA26'] = input_file['Close'].ewm(span=26).mean()
input_file['MACD']=input_file['EMA12']-input_file['EMA26']
input_file['EMA9M']=input_file['MACD'].ewm(span=9).mean()

In [8]:
input_file['MACDpos']=(input_file['MACD']>input_file['EMA9M']) & (input_file['EMA9M'].shift(1)>=input_file['MACD'].shift(1))
input_file['MACDneg']=(input_file['MACD']<input_file['EMA9M']) & (input_file['EMA9M'].shift(1)<=input_file['MACD'].shift(1))
input_file.tail()

,Open,High,Low,Close,Adj Close,Volume,SMA50,SMA10,pos_indicator,neg_indicator,SMA20,StdDev,UpperBand,LowerBand,BBpos,BBneg,EMA12,EMA26,MACD,EMA9M,MACDpos,MACDneg
Date,,,,,,,,,,,,,,,,,,,,,,
2019-12-23,12235.450195,12287.150391,12213.250000,12262.750000,12262.750000,600.0,11883.890996,12106.030078,False,False,12068.445117,122.474107,12313.393332,11823.496903,False,False,12135.139298,12036.540365,98.598933,79.906609,False,False
2019-12-24,12269.250000,12283.700195,12202.099609,12214.549805,12214.549805,500.0,11903.490996,12141.805078,False,False,12077.287598,126.456935,12330.201468,11824.373728,False,False,12147.356299,12049.726249,97.630050,83.451297,False,False
2019-12-26,12211.849609,12221.549805,12118.849609,12126.549805,12126.549805,500.0,11919.920996,12163.445020,False,False,12078.580078,126.840343,12332.260763,11824.899393,False,False,12144.155300,12055.416883,88.738417,84.508721,False,False
2019-12-27,12172.900391,12258.450195,12157.900391,12245.799805,12245.799805,400.0,11938.013984,12190.845020,False,False,12083.312549,131.375124,12346.062796,11820.562302,False,False,12159.792916,12069.519322,90.273594,85.661696,False,False
2019-12-30,12274.900391,12286.450195,12213.799805,12255.849609,12255.849609,400.0,11954.564980,12207.759961,False,False,12093.302539,136.682271,12366.667081,11819.937997,False,False,12174.570869,12083.321565,91.249304,86.779218,False,False


### Mean Return for the next 10 days

In [9]:
input_file['Sum']=0
for j in range(1,11):
    input_file['Sum']+=input_file['Close'].shift(-j)-input_file['Close']
input_file['mean_ret']=(input_file['Sum']/(input_file['Close']*10))
input_file.head()

,Open,High,Low,Close,Adj Close,Volume,SMA50,SMA10,pos_indicator,neg_indicator,SMA20,StdDev,UpperBand,LowerBand,BBpos,BBneg,EMA12,EMA26,MACD,EMA9M,MACDpos,MACDneg,Sum,mean_ret
Date,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,8288.700195,8410.599609,8288.700195,8395.450195,8395.450195,101900.0,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False,False,8395.450195,8395.450195,0.000000,0.000000,False,False,-919.651362,-0.010954
2015-01-05,8407.950195,8445.599609,8363.900391,8378.400391,8378.400391,118200.0,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False,False,8386.214884,8386.597412,-0.382528,-0.212515,False,True,-576.853518,-0.006885
2015-01-06,8325.299805,8327.849609,8111.350098,8127.350098,8127.350098,172800.0,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False,False,8285.179899,8293.452369,-8.272471,-3.515776,False,False,2501.898923,0.030784
2015-01-07,8118.649902,8151.200195,8065.450195,8102.100098,8102.100098,164100.0,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False,False,8227.388760,8239.958636,-12.569876,-6.582883,False,False,3381.798825,0.041740
2015-01-08,8191.399902,8243.500000,8167.299805,8234.599609,8234.599609,143800.0,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False,False,8229.347922,8238.715856,-9.367934,-7.411372,False,False,2583.604497,0.031375


In [10]:
keep_df=input_file[:-10][['Close','pos_indicator','neg_indicator','mean_ret','BBpos','BBneg','MACDpos','MACDneg']]
keep_df.tail()

,Close,pos_indicator,neg_indicator,mean_ret,BBpos,BBneg,MACDpos,MACDneg
Date,,,,,,,,
2019-12-09,11937.500000,False,False,0.014118,False,False,False,False
2019-12-10,11856.799805,False,False,0.024037,False,False,False,False
2019-12-11,11910.150391,False,False,0.021267,False,False,False,False
2019-12-12,11971.799805,False,False,0.018297,False,False,False,False
2019-12-13,12086.700195,False,False,0.010016,False,False,False,False


### Creating the bullish and bearish dataframes

In [11]:
bullish=pd.DataFrame()
bearish=pd.DataFrame()
bullish['mean_returns']=keep_df['mean_ret']
bearish['mean_returns']=keep_df['mean_ret']

In [12]:
bullish['SMAind']=[1 if keep_df.loc[ei,'pos_indicator'] 
                        else 0 for ei in keep_df.index]
bullish['BBind']=[1 if keep_df.loc[ei, 'BBpos']
                      else 0 for ei in keep_df.index]
bullish['MACDind']=[1 if keep_df.loc[ei, 'MACDpos']
                       else 0 for ei in keep_df.index]

bearish['SMAind']=[1 if keep_df.loc[ei,'neg_indicator']
                        else 0 for ei in keep_df.index]
bearish['BBind']=[1 if keep_df.loc[ei, 'BBneg']
                      else 0 for ei in keep_df.index]
bearish['MACDind']=[1 if keep_df.loc[ei, 'MACDneg']
                       else 0 for ei in keep_df.index]
print(bearish.tail())
print(bullish.tail())

            mean_returns  SMAind  BBind  MACDind
Date                                            
2019-12-09      0.014118       0      0        0
2019-12-10      0.024037       0      0        0
2019-12-11      0.021267       0      0        0
2019-12-12      0.018297       0      0        0
2019-12-13      0.010016       0      0        0
            mean_returns  SMAind  BBind  MACDind
Date                                            
2019-12-09      0.014118       0      0        0
2019-12-10      0.024037       0      0        0
2019-12-11      0.021267       0      0        0
2019-12-12      0.018297       0      0        0
2019-12-13      0.010016       0      0        0


### OLS linear regression

In [14]:
X=bearish['MACDind'] #independent variable
Y=bearish['mean_returns'] #dependent variable
X1=sm.add_constant(X)
reg=sm.OLS(Y,X1).fit()
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:           mean_returns   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                   0.07285
Date:                Sun, 19 Jul 2020   Prob (F-statistic):              0.787
Time:                        13:42:02   Log-Likelihood:                 3225.6
No. Observations:                1211   AIC:                            -6447.
Df Residuals:                    1209   BIC:                            -6437.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0019      0.000      3.923      0.0